In [1]:
import pandas as pd
import numpy as np
from numpy import pi, cos, sin, arcsin, sqrt


# Q1: Cargo freight scheduling

## Part 1: Building our data

In [2]:
# read data
cities = pd.read_csv('cargo-city-locations.csv')
# add index for each city
cities['Index'] = [i for i in range(len(cities))]
# display data
cities


,State,City,Latitude,Longitude,Index
0,Alabama,Montgomery,32.377716,-86.300568,0
1,Alaska,Juneau,58.301598,-134.420212,1
2,Arizona,Phoenix,33.448143,-112.096962,2
3,Arkansas,Little Rock,34.746613,-92.288986,3
4,California,Sacramento,38.576668,-121.493629,4
...,...,...,...,...,...
63,Washington,Seattle,47.606200,-122.332100,63
64,DC,Washington,38.907200,-77.036900,64
65,Texas,El Paso,31.761900,-106.485000,65
66,Michigan,Detroit,42.331400,-83.045800,66


In [3]:
# create empty dataframe with desired fields
distance = pd.DataFrame(columns=['State 1', 'City 1', 'Lat 1', 'Lon 1', 'Index 1',
                                 'State 2', 'City 2', 'Lat 2', 'Lon 2', 'Index 2']
                       )

# populate dataframe
for i in range(len(cities)):
    
    # extract info from current city
    state_1, city_1 = cities.loc[i, 'State'], cities.loc[i, 'City']
    lat_1, lon_1 = cities.loc[i, 'Latitude'], cities.loc[i, 'Longitude']
    idx_1 = cities.loc[i, 'Index']
    
    for j in range(len(cities)):
        
        # no need to map from a city to itself
        if i == j:
            continue
        
        state_2, city_2 = cities.loc[j, 'State'], cities.loc[j, 'City']
        lat_2, lon_2 = cities.loc[j, 'Latitude'], cities.loc[j, 'Longitude']
        idx_2 = cities.loc[j, 'Index']
        
        # insert row into new dataframe
        new_row = [state_1, city_1, lat_1, lon_1, idx_1, state_2, city_2, lat_2, lon_2, idx_2,]
        distance.loc[len(distance)] = new_row
    
# display data
distance
    

,State 1,City 1,Lat 1,Lon 1,Index 1,State 2,City 2,Lat 2,Lon 2,Index 2
0,Alabama,Montgomery,32.377716,-86.300568,0,Alaska,Juneau,58.301598,-134.420212,1
1,Alabama,Montgomery,32.377716,-86.300568,0,Arizona,Phoenix,33.448143,-112.096962,2
2,Alabama,Montgomery,32.377716,-86.300568,0,Arkansas,Little Rock,34.746613,-92.288986,3
3,Alabama,Montgomery,32.377716,-86.300568,0,California,Sacramento,38.576668,-121.493629,4
4,Alabama,Montgomery,32.377716,-86.300568,0,Colorado,Denver,39.739227,-104.984856,5
...,...,...,...,...,...,...,...,...,...,...
4551,Oregon,Portland,45.505100,-122.675000,67,North Carolina,Charlotte,35.227100,-80.843100,62
4552,Oregon,Portland,45.505100,-122.675000,67,Washington,Seattle,47.606200,-122.332100,63
4553,Oregon,Portland,45.505100,-122.675000,67,DC,Washington,38.907200,-77.036900,64
4554,Oregon,Portland,45.505100,-122.675000,67,Texas,El Paso,31.761900,-106.485000,65


In [4]:
# function to convert lats/lons to radian
def to_rad(deg):
    return (deg / 360) * (2*pi)

# apply conversion of lats and lons to radians
distance['Lat 1'] = distance['Lat 1'].apply(to_rad)
distance['Lon 1'] = distance['Lon 1'].apply(to_rad)
distance['Lat 2'] = distance['Lat 2'].apply(to_rad)
distance['Lon 2'] = distance['Lon 2'].apply(to_rad)

# compute haversine
distance['Haversine'] = ((1 - cos(distance['Lat 1'] - distance['Lat 2'])) / 2) + \
                        cos(distance['Lat 2']) * cos(distance['Lat 1']) * \
                        ((1 - cos(distance['Lon 1'] - distance['Lon 2'])) / 2)

# radius of earth
radius_earth = 6378.137
# compute distance from haversine
distance['Distance (km)'] = 2 * radius_earth * arcsin(sqrt(distance['Haversine']))

# speed of DC-10 aircraft in km / hr
speed = 908
# approximate travel time between cities
distance['Time (hrs)'] = distance['Distance (km)'] / speed

# display data
distance


,State 1,City 1,Lat 1,Lon 1,Index 1,State 2,City 2,Lat 2,Lon 2,Index 2,Haversine,Distance (km),Time (hrs)
0,Alabama,Montgomery,0.565098,-1.506229,0,Alaska,Juneau,1.017555,-2.346075,1,0.124070,4591.735528,5.056977
1,Alabama,Montgomery,0.565098,-1.506229,0,Arizona,Phoenix,0.583780,-1.956461,2,0.035199,2407.512376,2.651445
2,Alabama,Montgomery,0.565098,-1.506229,0,Arkansas,Little Rock,0.606443,-1.610747,3,0.002321,614.754613,0.677043
3,Alabama,Montgomery,0.565098,-1.506229,0,California,Sacramento,0.673290,-2.120464,4,0.063264,3243.340917,3.571961
4,Alabama,Montgomery,0.565098,-1.506229,0,Colorado,Denver,0.693580,-1.832331,5,0.021234,1865.471838,2.054484
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4551,Oregon,Portland,0.794214,-2.141083,67,North Carolina,Charlotte,0.614829,-1.410978,62,0.080987,3681.087606,4.054061
4552,Oregon,Portland,0.794214,-2.141083,67,Washington,Seattle,0.830885,-2.135098,63,0.000340,235.360927,0.259208
4553,Oregon,Portland,0.794214,-2.141083,67,DC,Washington,0.679059,-1.344548,64,0.085339,3781.614483,4.164774
4554,Oregon,Portland,0.794214,-2.141083,67,Texas,El Paso,0.554350,-1.858514,65,0.026131,2071.136208,2.280987


In [5]:
# check distances with known value
distance[(distance['City 1'] == 'Des Moines') & (distance['City 2'] == 'Baton Rouge')]


,State 1,City 1,Lat 1,Lon 1,Index 1,State 2,City 2,Lat 2,Lon 2,Index 2,Haversine,Distance (km),Time (hrs)
954,Iowa,Des Moines,0.725901,-1.633693,14,Louisiana,Baton Rouge,0.531576,-1.59152,17,0.009698,1258.226139,1.385712


(a) Which two cities have the highest travel travel time?

In [6]:
# compute max and display sample
max_time = distance['Time (hrs)'].max()
distance[distance['Time (hrs)'] == max_time]


,State 1,City 1,Lat 1,Lon 1,Index 1,State 2,City 2,Lat 2,Lon 2,Index 2,Haversine,Distance (km),Time (hrs)
553,Hawaii,Honolulu,0.371885,-2.755131,8,Maine,Augusta,0.773306,-1.217920,18,0.361898,8233.880502,9.06815
1214,Maine,Augusta,0.773306,-1.217920,18,Hawaii,Honolulu,0.371885,-2.755131,8,0.361898,8233.880502,9.06815


(b) Which two cities have the smallest travel time? 

In [7]:
# compute min and display sample
min_time = distance['Time (hrs)'].min()
distance[distance['Time (hrs)'] == min_time]


,State 1,City 1,Lat 1,Lon 1,Index 1,State 2,City 2,Lat 2,Lon 2,Index 2,Haversine,Distance (km),Time (hrs)
1996,New Jersey,Trenton,0.701982,-1.304981,29,Pennsylvania,Philadelphia,0.697304,-1.311880,54,0.000012,44.982011,0.04954
3647,Pennsylvania,Philadelphia,0.697304,-1.311880,54,New Jersey,Trenton,0.701982,-1.304981,29,0.000012,44.982011,0.04954


(c) Which city has the smallest average travel time to all of the other cities?

In [8]:
# dictionary to store times
avg_time_dict = {}

for city in distance['City 1']:
    
    # subset dataframe to have only the current city as an origin
    curr = distance[distance['City 1'] == city]
    # compute and record mean
    avg_time_dict[city] = curr['Time (hrs)'].mean()
    
# retrieve city with minimum average time
min(avg_time_dict, key=avg_time_dict.get)


'Springfield'

## Part 2: Finding a schedule

(a) Find the average travel time of a random trip to all cities using 100 simulations.

In [9]:
# set seed for reproducibility
np.random.seed(50)
# number of cities
n_cities = 68
# list to store total times
times =  [0 for k in range(100)]

# generate 100 random sequence of cities
for i in range(100):
    
    # generate sequence
    temp = np.random.permutation(n_cities)
    # track total time for the current trip
    total_time = 0
    
    # add up all times in the sequence
    for j in range(1, n_cities):
        
        # determine starting and ending city for the curent leg
        idx_from, idx_to = temp[j-1], temp[j]
        # calculate time
        time = distance[(distance['Index 1'] == idx_from) & (distance['Index 2'] == idx_to)]['Time (hrs)'].to_numpy()[0]
        # store time
        total_time += time
        
    # add travel time from last city to initial city
    start, end = temp[0], temp[-1]
    time = distance[(distance['Index 1'] == end) & (distance['Index 2'] == start)]['Time (hrs)'].to_numpy()[0]
    total_time += time
    
    # record total time
    times[i] = total_time
        
# display average time
np.mean(times)


149.4632186659759

(b) Suppose that we design the sequence of cities using the following heuristic. Starting from Los
Angeles, the next city in the schedule is the one that is closest to the current city in travel time
and has not been visited yet. What is the total travel time of this sequence, in hours?

In [10]:
# dataframe to manipulate as we go
remaining = distance.copy()
# start in Los Angeles
start_idx = distance[distance['City 1'] == 'Los Angeles']['Index 1'].to_numpy()[0]
# list of cities (indices) already visited
visited = [start_idx]
# remove Lost Angeles from remaining cities to visit
remaining = remaining[~remaining['Index 2'].isin(visited)]
# to track total time
total_time = 0

while len(set(visited)) < 68: # until 68 unique cities have been visited...
    
    # keep only cities that have the correct starting point
    temp_df = remaining[remaining['Index 1'] == start_idx]
    # find closest city
    min_val = temp_df['Time (hrs)'].min()
    temp_df = temp_df[temp_df['Time (hrs)'] == min_val]
    # extract time from that city
    time = temp_df['Time (hrs)'].to_numpy()[0]
    # add time to total time
    total_time += time
    # record that city as visited
    idx_end = temp_df['Index 2'].to_numpy()[0]
    visited.append(idx_end)
    # set that city to new start index
    start_idx = idx_end
    # remove that city from the destinations in the dataframe
    remaining = remaining[~remaining['Index 2'].isin(visited)]
    
# display total time
total_time


31.30938138161026

(c) Solve an optimization problem to find the order in which the cities should be visited, so as to
minimize the total travel time. What is the total travel time of this sequence, in hours?

In [11]:
# clean up distance dataframe
distance = distance[['State 1','City 1','Index 1','State 2','City 2','Index 2','Time (hrs)']]
# zip starting and ending indices into an array of tuples
start_end_zipped = list(zip(distance['Index 1'], distance['Index 2']))
# create dictionary with a time attached to each start-end pair
time_dict = dict(zip(start_end_zipped, distance['Time (hrs)']))


In [12]:
# function to retrieve subtours
def get_subtours(sequence):
    subtour_list = []
    unvisited = list(range(n_cities))
    
    while (len(unvisited) > 0):
        node = unvisited.pop()
        
        subtour = []
        subtour.append(node)
        
        next_node = list(filter(lambda t: t[0] == node, sequence))[0][1]
        
        while (next_node in unvisited):
            subtour.append(next_node)
            unvisited.remove(next_node)
            next_node = list(filter(lambda t: t[0] == next_node, sequence))[0][1]
            
        subtour_list.append(subtour)
    
    return subtour_list

# function to eliminate subtours via lazy constraints and ensure a valid tour of all cities
def eliminate_subtours(model, where):
    if (where == GRB.Callback.MIPSOL):
        x_val = model.cbGetSolution(x)
        sequence = [ (i,j) for (i,j) in od_pairs if x_val[i,j] > 0.5]
        subtour_list = get_subtours(sequence)
        if (len(subtour_list) > 1):
            for subtour in subtour_list:
                model.cbLazy( sum(x[i,j] for i in subtour for j in subtour if i != j) <= len(subtour) - 1)


In [13]:
from gurobipy import *

# we now solve the optimization problem

# initialize model
m = Model()

# number of cities (reinitialized to be safe)
n_cities = 68
# extract city pairs
od_pairs = time_dict.keys()

# decision variables
x = m.addVars(od_pairs, vtype=GRB.BINARY)

# constraints
for i in range(n_cities):
    # go to only one city from city i
    m.addConstr(sum(x[i,j] for j in range(n_cities) if j != i ) == 1)
    # arrive at city i from only one city
    m.addConstr( sum(x[j,i] for j in range(n_cities) if j != i ) == 1)
    
# objective function
m.setObjective(sum(time_dict[i,j] * x[i,j] for (i,j) in od_pairs), GRB.MINIMIZE)
# update model
m.update()

# enable lazy constraints
m.params.LazyConstraints = 1
# supply callback to Gurobi and optmiize
m.optimize(eliminate_subtours)



--------------------------------------------
--------------------------------------------

Using license file C:\Users\HP\gurobi.lic
Academic license - for non-commercial use only - expires 2021-05-29
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 136 rows, 4556 columns and 9112 nonzeros
Model fingerprint: 0x5c4ba032
Variable types: 0 continuous, 4556 integer (4556 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 136 rows, 4556 columns, 9112 nonzeros
Variable types: 0 continuous, 4556 integer (4556 binary)

Root relaxation: objective 2.729102e+01, 163 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Wor

In [14]:
# display total travel time
m.ObjVal


30.553624275955524

In [15]:
# display actual tour of cities
sequence = [(i,j) for (i,j) in od_pairs if x[i,j].x > 0.5]
subtour_list = get_subtours(sequence)
complete_tour = subtour_list[0]
complete_tour.append(complete_tour[0])
complete_tour_names = []
for idx in complete_tour:
    # convert city indices to names
    city = cities[cities['Index'] == idx]['City'].to_numpy()[0]
    complete_tour_names.append(city)
print("Complete tour: ", complete_tour_names)


Complete tour:  ['Portland', 'Olympia', 'Seattle', 'Juneau', 'Honolulu', 'San Francisco', 'San Jose', 'Sacramento', 'Carson City', 'Los Angeles', 'San Diego', 'Phoenix', 'El Paso', 'Santa Fe', 'Oklahoma City', 'Dallas', 'Fort Worth', 'Austin', 'San Antonio', 'Houston', 'Baton Rouge', 'Jackson', 'Little Rock', 'Jefferson City', 'Topeka', 'Lincoln', 'Des Moines', 'Springfield', 'Indianapolis', 'Frankfort', 'Nashville', 'Atlanta', 'Montgomery', 'Tallahassee', 'Jacksonville', 'Columbia', 'Charlotte', 'Raleigh', 'Richmond', 'Washington', 'Annapolis', 'Dover', 'Philadelphia', 'Trenton', 'New York City', 'Hartford', 'Providence', 'Boston', 'Concord', 'Augusta', 'Montpelier', 'Albany', 'Harrisburg', 'Charleston', 'Columbus', 'Detroit', 'Lansing', 'Chicago', 'Madison', 'St. Paul', 'Bismarck', 'Pierre', 'Cheyenne', 'Denver', 'Salt Lake City', 'Helena', 'Boise', 'Salem', 'Portland']


# Q2: Winning the NBA championship

See R script for Part 1 and Part 2

## Part 3: A dream team

In [16]:
# read data
nba_players = pd.read_csv('nba-players-2018-2019-with-pos.csv')
nba_players.head()


,Player,Pos,Tm,X3PA,X2PA,FGA,FTA,ORB,DRB,AST,STL,BLK,TOV,PF,Salary
0,Alex Abrines,SG,OKC,127,30,157,13,5,43,20,17,6,14,53,5455236.0
1,Quincy Acy,PF,PHO,15,3,18,10,3,22,8,1,4,4,24,213949.0
2,Jaylen Adams,PG,ATL,74,36,110,9,11,49,65,14,5,28,45,236854.0
3,Steven Adams,C,OKC,2,807,809,292,391,369,124,117,76,135,204,24157304.0
4,Bam Adebayo,C,MIA,15,471,486,226,165,432,184,71,65,121,203,2955840.0


In [17]:
# coefficients from the regression we ran in R
x0 = 513.82999 
X3PA = 0.69129
X2PA = 0.36184 
FTA = 0.78122
ORB = -0.20277 
DRB = 0.61064
AST = 0.91510
STL = 0.50506
BLK = 0.07975
TOV = -0.62701
PF = 0.30420
# pack coefficients into vector
coefs = np.array([X3PA,X2PA,FTA,ORB,DRB,AST,STL,BLK,TOV,PF])
# predict points on a player-by-player basis
nba_players['PTS'] = nba_players.drop(columns=['Player', 'Pos', 'Tm', 'Salary', 'FGA']).dot(coefs)
# display data
nba_players.head()


,Player,Pos,Tm,X3PA,X2PA,FGA,FTA,ORB,DRB,AST,STL,BLK,TOV,PF,Salary,PTS
0,Alex Abrines,SG,OKC,127,30,157,13,5,43,20,17,6,14,53,5455236.0,168.75954
1,Quincy Acy,PF,PHO,15,3,18,10,3,22,8,1,4,4,24,213949.0,45.03046
2,Jaylen Adams,PG,ATL,74,36,110,9,11,49,65,14,5,28,45,236854.0,161.98738
3,Steven Adams,C,OKC,2,807,809,292,391,369,124,117,76,135,204,24157304.0,823.58266
4,Bam Adebayo,C,MIA,15,471,486,226,165,432,184,71,65,121,203,2955840.0,782.99694


Constraints:

* Team should consist of exactly 15 players.
* The total compensation paid to the team should not exceed $100 million. (The file
nba-players-2018-2019-with-pos.csv contains salary information for each player in the
2018-2019 season.)
* There should be exactly three players in each of the following positions: C (center); PF (power
forward); SF (small forward); SG (shooting guard); and PG (shooting guard). (The file
nba-players-2018-2019-with-pos.csv contains position information for all of the players.)

Variables:
* $x_i = 1$ if player $i$ is in the dream team, $0$ otherwise
* $p_i =$ expected points from player $i$
* $s_i$ = salary of player $i$
* $pos_{i,j} = 1$ if player $i$ plays position $j$, $0$ otherwise

Formulation:
${max}_x\sum_{i=1}^n x_i p_i$ subject to:
1. $\sum_{i=1}^n x_i = 15$ (team must consist of 15 players)
2. $\sum_{i=1}^n x_i s_i \leq 100,000,000$ (total salary cap is 100M)
3. $\sum_{i=1}^n x_i pos_{i,j} = 3$ for $j = 1, 2,..., 5$ (must have 3 players in each position)

In [18]:
# create salary array
s = np.array(nba_players['Salary'])
# create position array
p = np.array(pd.get_dummies(nba_players['Pos']))
# create points array
pts = np.array(nba_players['PTS'])


In [19]:
# initialize model
m_nba = Model()
# number of players
n_players = len(nba_players)
# size of team
team_size = 15
# max total salary
salary_cap = 100_000_000
# number of positions
n_pos = 5
# number of players needed in each position
pos_need = 3

# initialize decision variables
x = m_nba.addVars(n_players, vtype=GRB.BINARY)

# cosntraint on number of players
m_nba.addConstr(sum(x[i] for i in range(n_players)) == team_size)
# constraint on total salary
m_nba.addConstr(sum(x[i] * s[i] for i in range(n_players)) <= salary_cap)
# constraint on positions
for j in range(n_pos):
    # must have 3 in each position
    m_nba.addConstr(sum(x[i] * p[i,j] for i in range(n_players)) == pos_need)
    
# objective function (maximize total points scored), include baseline points per team (x0)
m_nba.setObjective(sum(x[i] * pts[i] for i in range(n_players)) + x0, GRB.MAXIMIZE)
# update model
m_nba.update()
# solve the model
m_nba.optimize()


Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 7 rows, 522 columns and 1566 nonzeros
Model fingerprint: 0xa2426708
Variable types: 0 continuous, 522 integer (522 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+07]
  Objective range  [2e-01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+08]
Found heuristic solution: objective 6837.4563700
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 522 columns, 1044 nonzeros
Variable types: 0 continuous, 522 integer (522 binary)

Root relaxation: objective 2.069517e+04, 19 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 20695.1697    0    2 6837.45637 20695.1697   203%     -    0s
H    0     0                    20486.301820 20695.1697  1

In [20]:
# list to store dream team indices
dream_team_indices = []
for i in range(n_players):
    if x[i].x != 0:
        dream_team_indices.append(i)
        
# display dream team
nba_players.loc[dream_team_indices]
        

,Player,Pos,Tm,X3PA,X2PA,FGA,FTA,ORB,DRB,AST,STL,BLK,TOV,PF,Salary,PTS
17,Giannis Antetokounmpo,PF,MIL,203,1044,1247,686,159,739,424,92,110,268,232,24157304.0,1818.80842
59,Devin Booker,SG,PHO,414,841,1255,454,39,226,433,56,13,264,200,3314365.0,1396.13976
138,Luka Doncic,SG,DAL,514,672,1186,485,86,477,429,77,25,247,137,6560640.0,1571.47350
167,De'Aaron Fox,PG,SAC,232,870,1102,417,43,261,590,133,45,227,204,5470920.0,1482.00301
286,Kyle Kuzma,PF,LAL,422,665,1087,250,60,322,178,41,26,133,170,1689840.0,1066.10329
347,Donovan Mitchell,SG,UTA,519,1011,1530,396,59,257,322,106,31,218,208,3111480.0,1456.19015
384,Cedi Osman,SF,CLE,374,470,844,181,45,312,195,60,11,114,195,2775000.0,948.86832
427,Domantas Sabonis,C,IND,17,683,700,291,186,504,212,48,30,160,239,2659800.0,949.28979
439,Pascal Siakam,PF,TOR,214,731,945,302,124,425,248,73,52,154,241,1544951.0,1127.46190
457,Jayson Tatum,SF,BOS,311,725,1036,228,70,407,168,84,57,122,168,6700800.0,1065.09790


(a) Which players should you select for your team? Explain your approach.

I would choose the above players for my dream team. My approach involved predicting points per player by using a dot product of the coefficients from the regression I ran in R with the stats from each player (since the regression used entire teams) and adding the baseline points per team (i.e. the intercept) to my dream team at the end. I then set up an optimization model to maximize the expected number of points scored by my team. To formulate this optimization problem, I created arrays of players to select (x), salaries (s), positions (p), and points (pts), and introduced the appropriate constraints on those arrays. I then set my objective function to be the total number of points across the whole team.

(b) What is the predicted number of points that your team will score in the upcoming season?

In [21]:
nba_players.loc[dream_team_indices]['PTS'].sum()

20063.80805

(c) What are some limitations to our analysis in (a) - (b)? For the purpose of building an "optimal"
team, what are some ways that you might improve the predictive model and/or optimization
approach?

One of the main limitations of our analysis in (a) and (b) is that it only optimizes on points scored. However, there potentially may be other aspects of the game that are desirable beyond points scored. To incorporate this into our model, we could make the objective function depend on variables like rebounds (ORB and DRB) in order to achieve a more balanced dream team. Another limitation is that the expected points for each player is predicted and thus we aren't sure how much we can trust these values. To address this, we could try out several regression approaches in addition to the linear regression that we used to see if we can improve these predictions.